In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
def create_spark_connection():
    s_conn = None

    try:
        s_conn = SparkSession.builder \
            .appName('SparkDataStreaming') \
            .config('spark.jars.packages', "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
            .getOrCreate()

        # s_conn.sparkContext.setLogLevel("ERROR")
        print("Spark connection created successfully!")
    except Exception as e:
        print(f"Couldn't create the spark session due to exception {e}")

    return s_conn

In [ ]:
spark_conn = create_spark_connection()

In [4]:
def connect_to_kafka(spark_conn):
    spark_df = None
    try:
        spark_df = spark_conn.readStream \
            .format('kafka') \
            .option('kafka.bootstrap.servers', 'kafka:9092') \
            .option('subscribe', 'test') \
            .option('startingOffsets', 'earliest') \
            .load()
        print("kafka dataframe created successfully")
    except Exception as e:
        print(f"kafka dataframe could not be created because: {e}")

    return spark_df


In [5]:
spark_df = connect_to_kafka(spark_conn)

kafka dataframe created successfully


In [6]:
spark_df.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
query = spark_df \
    .writeStream \
    .outputMode('append') \
    .format('console') \
    .start()

# query.awaitTermination()

23/11/14 15:22:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-579666a1-c854-4d50-9edc-475583c0deb0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/14 15:22:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/11/14 15:22:57 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|NULL|          [62 6A 72]| test|        0|     0|2023-11-14 14:49:...|            0|
|NULL|       [61 6C 6C 6F]| test|        0|     1|2023-11-14 14:50:...|            0|
|NULL|[6E 6A 6E 7A 6F 6...| test|        0|     2|2023-11-14 14:57:...|            0|
|NULL|[6E 6A 6E 7A 6F 6...| test|        0|     3|2023-11-14 14:59:...|            0|
|NULL|[6E 6A 6E 7A 6F 6...| test|        0|     4|2023-11-14 15:01:...|            0|
|NULL|[6E 6A 6E 7A 6F 6...| test|        0|     5|2023-11-14 15:03:...|            0|
|NULL| [6F 68 6F 68 6F 68]| test|        0|     6|2023-11-14 15:08:...|            0|
+----+--------------------+-----+---------+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+-------------------+-----+---------+------+--------------------+-------------+
| key|              value|topic|partition|offset|           timestamp|timestampType|
+----+-------------------+-----+---------+------+--------------------+-------------+
|NULL|[61 68 61 68 61 68]| test|        0|     7|2023-11-14 15:23:...|            0|
+----+-------------------+-----+---------+------+--------------------+-------------+

